In [2]:
import scapy
from scapy.all import *

import pandas as pd

packets=rdpcap('../data/web.pcap')

In [107]:
import numpy as np

In [3]:
print(len(packets))

122760


### Features to extract

    1. FW Packet length max
    flow duration
    flow packets/s
    fwd IAt total
    flow IAT std
    Fwd packet length std
    fwd packet length mean
    total length of backward packets
    flow bytes/s
    total length of fwd packets
    flow iat mean
    Bot(Bwd Packet Length Mean)
    DDos(Bwd Packet Length Std)
    DDos golden eye (Flow IAT Max)
    Total Fwd Packets
    Total Backward Packets'

In [74]:
list_of_all_dictionaries=[]

for i in range(len(packets)):

    dictionary_of_data={}
    pkt=packets[i]

    # Features from IP
    try :
        dictionary_of_data['sender_ip']=pkt['IP'].src
        dictionary_of_data['destination_ip']=pkt['IP'].dst
        dictionary_of_data['protocol_ip']=pkt['IP'].proto
        dictionary_of_data['length']=pkt['IP'].len
    except:
        continue# Features from TCP

    try:
        dictionary_of_data['source_port']=pkt['TCP'].sport
        dictionary_of_data['destination_port']=pkt['TCP'].dport
        dictionary_of_data['data_of_s']=pkt['TCP'].dataofs
        dictionary_of_data['seq']=pkt['TCP'].seq
        dictionary_of_data['ack']=pkt['TCP'].ack
        dictionary_of_data['flags']=pkt['TCP'].flags
        dictionary_of_data['window']=pkt['TCP'].window
        dictionary_of_data['chksum']=pkt['TCP'].chksum
        dictionary_of_data['urgptr']=pkt['TCP'].urgptr
        dictionary_of_data['start_time']=pkt['TCP'].options[2][1][1]
        dictionary_of_data['end_time']=pkt['TCP'].options[2][1][0]
    except:
        continue
#     elif pkt['IP'].proto==17:
#         try:
#             dictionary_of_data['source_port']=pkt['UDP'].sport
#             dictionary_of_data['destination_port']=pkt['UDP'].dport
#             dictionary_of_data['chksum']=pkt['UDP'].chksum
#         except:
#             continue

    list_of_all_dictionaries.append(dictionary_of_data)# Only store data if TCP data is available


    




print(len(list_of_all_dictionaries))


95433


In [121]:
packets[30]

<CookedLinux  pkttype=sent-by-us lladdrtype=0x1 lladdrlen=6 src='\xd4n\x0e\n\xbb\xb3' proto=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=52 id=22049 flags=DF frag=0 ttl=64 proto=tcp chksum=0x3477 src=192.168.8.100 dst=13.76.217.211 |<TCP  sport=54180 dport=https seq=364702522 ack=3869134729 dataofs=8 reserved=0 flags=A window=501 chksum=0xa1 urgptr=0 options=[('NOP', None), ('NOP', None), ('Timestamp', (2653982487, 1386035095))] |>>>

In [75]:
df=pd.DataFrame(list_of_all_dictionaries)

In [76]:
df.to_csv('../data/web.csv')

In [77]:
df.head()

,sender_ip,destination_ip,protocol_ip,length,source_port,destination_port,data_of_s,seq,ack,flags,window,chksum,urgptr,start_time,end_time
0,192.168.8.100,13.76.217.211,6,60,54180,443,10,364700558,0,S,64240,41692,0,0,2653981995
1,192.168.8.100,13.76.217.211,6,52,54180,443,8,364700559,3869128378,A,502,9133,0,1386034759,2653982164
2,192.168.8.100,13.76.217.211,6,569,54180,443,8,364700559,3869128378,PA,502,24744,0,1386034759,2653982164
3,13.76.217.211,192.168.8.100,6,1500,443,54180,8,3869128378,364701076,A,1029,52073,0,2653982164,1386034926
4,192.168.8.100,13.76.217.211,6,52,54180,443,8,364701076,3869129826,A,501,6847,0,1386034926,2653982319


#### Fwd Packet Length Max , Flow Duration

In [122]:
sessions = []
data = {}
time_data = {}
data_size = {}
for row in df.iterrows():
    
    address = row[1]['sender_ip'] + row[1]['destination_ip']
    address2 = row[1]['destination_ip'] + row[1]['sender_ip']
    
    if row[1]['flags'] == 'S':  # SYN frst step in conection setup
        sessions.append(address)
        data[address] = []
        time_data[address] = []
        data_size[address] = 0
        
    if row[1]['flags'] == 'A' or row[1]['flags'] == 'PA':
        
        
        if address in sessions: # forward
            
            if len(data[address])<1: # start time
                time_data[address].append(row[1]['start_time'])

            value = row[1]['length']
            data[address].append(value)
            
            data_size[address] = data_size[address] + row[1]['data_of_s']
            
        if address2 in sessions: # Backward
            data_size[address2] = data_size[address2] + row[1]['data_of_s']

            
    if row[1]['flags'] == 'F' or row[1]['flags'] == 'FA':
        
        if address in sessions:
            time_data[address].append(row[1]['end_time'])
            data_size[address] = data_size[address] + row[1]['data_of_s']

        elif address2 in sessions:
            time_data[address2].append(row[1]['end_time'])
            data_size[address2] = data_size[address2] + row[1]['data_of_s']

        

            
        
    


In [106]:
data

{'192.168.8.10013.76.217.211': [52,
  569,
  64,
  64,
  64,
  64,
  52,
  178,
  145,
  1279,
  52,
  90,
  52,
  52,
  52],
 '192.168.8.10068.232.34.200': [52, 52, 52, 52, 52, 52, 52, 52, 52, 52],
 '192.168.8.100140.82.114.3': [52,
  569,
  52,
  52,
  52,
  52,
  52,
  569,
  64,
  64,
  52,
  116,
  1476,
  1476,
  385,
  52,
  52,
  52,
  116,
  52,
  1476,
  1476,
  490,
  569,
  64,
  64,
  52,
  116,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  64,
  64,
  52,
  116,
  52,
  64,
  52,
  76,
  76,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  76,
  52,
  52,
  76,
  76],
 '192.168.8.100151.101.120.133': [52,
  52,
  52,
  52,
  52,
  569,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  64,
  52,
  145,
  436,
  64,
  52,
  145,
  52,
  52,
  52,
  52,
  52,
  145,
  437,
  52,
  145,
  436,
  436,
  52,
  145,
  52,
  64,
  52,
  64,
  52,
  145,
  437,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  52,
  

In [ ]:
# Forward Packet length max

for key, values in data.items():
    if len(values)>0:
        print(np.max(values))

In [116]:
time_data

{'192.168.8.10013.76.217.211': [1388484600],
 '192.168.8.10068.232.34.200': [1424969700],
 '192.168.8.100140.82.114.3': [1693155916,
  2395855254,
  2395855254,
  1677825274,
  1690034703,
  2395909752,
  1695811796,
  1691614704,
  2395910369,
  1693171097,
  2395910422],
 '192.168.8.100151.101.120.133': [3640915010,
  3772110471,
  3772114508,
  3772114675,
  4218657946,
  3772117422,
  2539201414,
  1088766068,
  3772235343,
  2538403165,
  3772303362,
  3640965382],
 '192.168.8.100172.217.19.19': [845017717, 2187356998, 845319593],
 '192.168.8.10035.222.85.5': [595378190, 2080742460, 595387334],
 '192.168.8.100140.82.112.26': [2693145029,
  4268742309,
  3797129130,
  3157844404,
  4268764916,
  3797150492],
 '192.168.8.10035.224.99.156': [1725105384, 1725105745, 1140365460],
 '192.168.8.100157.240.196.35': [1333468902, 4264707256, 1333591705],
 '192.168.8.100104.103.245.50': [1214284640,
  2512234352,
  1214290821,
  2512730141,
  2512732142],
 '192.168.8.10034.194.150.218': [1194

In [124]:
# Duration
flow_duration = {}
for key, values in time_data.items():
    if len(values)>1:
        flow_duration[key] = values[-1]-values[0]
        

In [125]:
# Flow bytes per second
flow_bytes_per_second = {}
for key, values in data_size.items():
        if key in flow_duration:
            flow_bytes_per_second[key] = (values/flow_duration[key]) * 8

In [126]:
flow_bytes_per_second

{'192.168.8.100140.82.114.3': 1.3512542315879509e-05,
 '192.168.8.100151.101.120.133': 0.3441594536647344,
 '192.168.8.100172.217.19.19': 0.040201937219255586,
 '192.168.8.10035.222.85.5': 0.07699037620297462,
 '192.168.8.100140.82.112.26': 2.768102244436086e-06,
 '192.168.8.10035.224.99.156': -9.850533140610388e-07,
 '192.168.8.100157.240.196.35': 0.02807748996360024,
 '192.168.8.100104.103.245.50': 1.736843419950605e-05,
 '192.168.8.10034.194.150.218': 0.07600263810809962,
 '192.168.8.10013.35.183.74': 7.775054324206867,
 '192.168.8.10093.184.220.29': -1.2451230713387703e-05,
 '192.168.8.100216.58.208.243': 0.023659198032491273,
 '192.168.8.100182.176.154.42': 1.4198374822649298,
 '192.168.8.100172.217.19.170': 0.013539074166251866,
 '192.168.8.10013.35.183.10': 0.002986000348274204,
 '192.168.8.10064.233.184.81': 1.6242727501313872e-06,
 '192.168.8.100216.58.207.3': -1.0198769745476656e-05,
 '192.168.8.100172.217.19.174': 0.5313463928377555,
 '192.168.8.100216.58.208.238': 0.1057400